In [1]:
#!/usr/bin/env python
# # -*- coding: utf-8 -*-
# """
# Created on Sat Jun 23 19:15:35 2024
# @author: Mazhar
# """

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
pwd

'/content'

In [4]:
try:
    import geopandas as gpd
    import pandas as pd
    import numpy as np
    from pykrige.ok import OrdinaryKriging
    from sklearn.neighbors import KNeighborsClassifier
    from tqdm import tqdm
    import sqlite3
    import os
    import sys
except ModuleNotFoundError:
    if 'google.colab' in str(get_ipython()):
        !apt install libspatialindex-dev -qq
        !pip install fiona shapely pyproj rtree --quiet
        !pip install geopandas --quiet
        !pip install pykrige --quiet
        !pip install scikit-learn
        !pip install tqdm --quiet
        !pip install pysqlite3 --quiet
        !pip install pandas --quiet
        !pip install numpy --quiet
        !pip install osmnx --quiet
        !pip install pyproj --quiet
        !pip install rtree --quiet
        !pip install fiona --quiet
        !pip install shapely --quiet
    else:
        print('geopandas not found, please install via conda in your environment')

The following additional packages will be installed:
  libspatialindex-c6 libspatialindex6
The following NEW packages will be installed:
  libspatialindex-c6 libspatialindex-dev libspatialindex6
0 upgraded, 3 newly installed, 0 to remove and 45 not upgraded.
Need to get 319 kB of archives.
After this operation, 1,416 kB of additional disk space will be used.
Selecting previously unselected package libspatialindex6:amd64.
(Reading database ... 121925 files and directories currently installed.)
Preparing to unpack .../libspatialindex6_1.9.3-2_amd64.deb ...
Unpacking libspatialindex6:amd64 (1.9.3-2) ...
Selecting previously unselected package libspatialindex-c6:amd64.
Preparing to unpack .../libspatialindex-c6_1.9.3-2_amd64.deb ...
Unpacking libspatialindex-c6:amd64 (1.9.3-2) ...
Selecting previously unselected package libspatialindex-dev:amd64.
Preparing to unpack .../libspatialindex-dev_1.9.3-2_amd64.deb ...
Unpacking libspatialindex-dev:amd64 (1.9.3-2) ...
Setting up libspatialindex6:a

In [5]:
import geopandas as gpd
import pandas as pd
import numpy as np
from pykrige.ok import OrdinaryKriging
from sklearn.neighbors import KNeighborsClassifier
from tqdm import tqdm
import osmnx as ox
import pyproj
import sqlite3
import os
import sys

In [6]:
ZONE = 4

PREFIX = f"/content/drive/MyDrive/IPAUA_Maz/dataset/"

FILE_1 = f"{PREFIX}zone_4_v2.csv"
FILE_2 = f"{PREFIX}zone_9_v2.csv"

# SAMPLE_FRACTION = 0.0028
SAMPLE_FRACTION = 0.05
LAYER_NAME = "spatial_Joins"

SAVE_FILE_CSV = f"{PREFIX}/combined_v2.csv"
SAVE_FILE_GPKG = f"{PREFIX}/combined_v2.gpkg"

In [7]:
df4 = pd.read_csv(FILE_1)
df9 = pd.read_csv(FILE_2)

In [8]:
# Add the 'Zone' column with a fixed value 'zone4'
df4['Zone'] = 4
df9['Zone'] = 9

In [9]:
# Move the geometry column to the last position
columns = [col for col in df4.columns if col != 'geometry'] + ['geometry']
df4 = df4[columns]

# Move the geometry column to the last position
columns = [col for col in df9.columns if col != 'geometry'] + ['geometry']
df9 = df9[columns]

In [10]:
df9.head()

,soil_moisture,NDBI,BU,Roughness,Slope,NDVI,LST,UHI,UTFVI,NDWI,...,CH4,CO,HCHO,NO2,O3,SO2,Longitude,Latitude,Zone,geometry
0,45.511588,-0.034360,-0.088197,52,89.990234,0.205030,32.807213,-0.024135,-0.001928,-0.197248,...,1907.709900,0.034171,0.000119,0.000147,0.141926,0.000077,9.192370,45.480355,9,POINT (9.192370471 45.48035535999999)
1,41.233572,-0.034360,-0.088197,52,89.990234,0.205030,32.807213,-0.024135,-0.001928,-0.273191,...,1906.761955,0.034172,0.000119,0.000147,0.141923,0.000076,9.192550,45.480355,9,POINT (9.192550134 45.48035536)
2,60.945201,-0.052701,-0.134789,1,89.986189,0.289858,32.765621,-0.040007,-0.003200,-0.293753,...,1906.761685,0.034171,0.000119,0.000147,0.141923,0.000076,9.192730,45.480355,9,POINT (9.192729797 45.48035535999999)
3,60.024595,-0.052701,-0.134789,1,89.984559,0.123400,35.511621,1.007879,0.074375,-0.157025,...,1906.761315,0.034171,0.000119,0.000147,0.141923,0.000076,9.192909,45.480355,9,POINT (9.19290946 45.48035536)
4,42.472361,-0.009582,-0.067800,1,89.984559,0.123400,35.511621,1.007879,0.074375,-0.255411,...,1906.760851,0.034171,0.000119,0.000147,0.141923,0.000076,9.193089,45.480355,9,POINT (9.193089123 45.48035535999999)


In [ ]:
# Check if the number of columns is the same
if df4.shape[1] == df9.shape[1]:
    print("Both DataFrames have the same number of columns.")
else:
    print("DataFrames do not have the same number of columns.")

Both DataFrames have the same number of columns.


In [ ]:
# Check if the column names match
if list(df4.columns) == list(df9.columns):
    print("Both DataFrames have the same column names in the same order.")
else:
    print("DataFrames do not have the same column names or order.")

Both DataFrames have the same column names in the same order.


In [ ]:
# Check if the number of columns and column names match
if df4.shape[1] == df9.shape[1] and list(df4.columns) == list(df9.columns):
    # Concatenate DataFrames vertically
    concatenated_df = pd.concat([df4, df9], axis=0)
    # Reset index to have a continuous index and drop the old index
    concatenated_df.reset_index(drop=True, inplace=True)
    print(concatenated_df)
else:
    print("DataFrames do not have the same number of columns or column names.")

         soil_moisture      NDBI        BU  Roughness      Slope      NDVI  \
0           125.331552 -0.129746 -0.271251        0.0  89.999694  0.545336   
1           130.649125 -0.091416 -0.238252        0.0  89.999691  0.705981   
2           130.643567 -0.112200 -0.245295        0.0  89.999691  0.705981   
3           195.133153 -0.129746 -0.271251        0.0  89.999694  0.545336   
4           132.716402 -0.129746 -0.271251        0.0  89.999694  0.545336   
...                ...       ...       ...        ...        ...       ...   
1973356      43.280261 -0.120166 -0.301888        0.0  89.992510  0.629332   
1973357      40.262151 -0.120166 -0.301888        0.0  89.992510  0.629332   
1973358      39.394996 -0.125362 -0.305908        0.0  89.986189  0.625278   
1973359      39.004049 -0.125362 -0.305908        0.0  89.969118  0.630991   
1973360      50.393403 -0.120438 -0.247285        0.0  89.969118  0.630991   

               LST       UHI     UTFVI      NDWI  ...          

#### SAVING TO .CSV

In [ ]:
# Save the merged GeoDataFrame to a CSV file
print(f"DATA IS BEING SAVED TO .CSV FILE ........Please Wait")
concatenated_df.to_csv(SAVE_FILE_CSV, index=False)
print(f"DATA SAVE TO {SAVE_FILE_CSV}")

# Get the file size in bytes
file_size_bytes = os.path.getsize(SAVE_FILE_CSV)
# Convert the file size to megabytes
file_size_mb = file_size_bytes / (1024 * 1024)
print(f"File Size of {SAVE_FILE_CSV}: {file_size_mb:.2f} MB")

DATA IS BEING SAVED TO .CSV FILE ........Please Wait
DATA SAVE TO /content/drive/MyDrive/IPAUA_Maz/dataset//combined_v2.csv
File Size of /content/drive/MyDrive/IPAUA_Maz/dataset//combined_v2.csv: 717.65 MB


#### SAVING TO .GPKG

In [ ]:
# Save to GPKG file
print(f"DATA IS BEING SAVED TO .GPKG FILE ........Please Wait")
gdf = gpd.GeoDataFrame(concatenated_df, geometry=gpd.points_from_xy(concatenated_df["Longitude"], concatenated_df["Latitude"]), crs="EPSG:4326")
gdf.to_file(driver='GPKG', filename=SAVE_FILE_GPKG, layer=LAYER_NAME)
print(f"DATA SAVED TO {SAVE_FILE_GPKG}")

# Get the file size in bytes
file_size_bytes = os.path.getsize(SAVE_FILE_GPKG)
# Convert the file size to megabytes
file_size_mb = file_size_bytes / (1024 * 1024)
print(f"File Size of {SAVE_FILE_GPKG}: {file_size_mb:.2f} MB")

DATA IS BEING SAVED TO .GPKG FILE ........Please Wait
DATA SAVED TO /content/drive/MyDrive/IPAUA_Maz/dataset//combined_v2.gpkg
File Size of /content/drive/MyDrive/IPAUA_Maz/dataset//combined_v2.gpkg: 571.61 MB


In [ ]:
# concatenated_df.head()

In [ ]:
# gdf.head()

In [ ]:
# gdf.tail()